# Implementación de los métodos




En este notebook se plantea la solución del problema utilizando los algoritmos planteados. Este archivo está autocontenido, sin embargo, la implementación principal se realiza con enfoque modular.

A continuación se describe el planteamiento del problema, y su implementación con el set de datos utilizado. Una explicación más detallada se realiza en el informe (en formato PDF) de este proyecto.


**Nota:** Esta implementación se basa en material y actividades impartidas por los profesores de los cursos de [Métodos Numéricos y optimización](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/IV.optimizacion_convexa_y_machine_learning/4.3.Regresion_logistica_R.ipynb) (2010-I) (Prof. Erick Palacios Moreno) y Aprendizaje de Máquina (2019-II) (Prof. Rodrigo Mendoza Smith).


## Dataset


Inicialmente, se considera un conjunto de datos que incorpora una variable output, $y_{i}$ asociada a la supervivencia o no del paciente ${i}$ con virus del ébola, y ${j}$ variables explicativas asociadas, $x_{i,j}$. Los regresores escogidos son aquellos que, conforme con nuestra principal referencia, son buenos predictores de la probabilidad de muerte o no de un paciente.

|Tipo| Nombre|Descripción|
|--- | --- | --- |
Variable Numérica| CT |El cycle threshold (CT) es una variable que se calcula a partir de una relación médica bien conocida (qPCR) y la carga viral (una expresión númerica de la cantidad de virus dado un volúmen de fluido que normalmente se correlaciona con la severidad de una infección viral activa).|
Variable Numérica|TEMP|Temperatura corporal del paciente.|
Variable Numérica|_AGE_ |Edad del paciente.|
Variable Categórica |_HEADCH_ | Presencia o no dolores de cabeza. Toma valores valores $0$ o $1$, dependiendo de si el paciente presenta o no dolores de cabeza.|
Variable Categórica |  _BLEED_ | Presencia o no de sangrado. Toma valores valores $0$ o $1$, dependiendo de si el paciente presenta o no sangrado. |
Variable Categórica |  _DIARR_ | Presencia o no de diarrea. Toma valores valores $0$ o $1$, dependiendo de si el paciente presenta o no diarrea.|
Variable Categórica | _VOMIT_ | Presencia o no de vómitos. Toma valores valores $0$ o $1$, dependiendo de si el paciente presenta o no vómito.|
Variable Categórica | _PABD_ | Presencia o no de PADB.
Variable Categórica |_WEAK_ | Presencia o no de debilidad o fatiga general.|
Variable Categórica |_JAUN_ |Condición  en la cuál la piel, los ojos y los miembros mucosos que vuelven amarillos debido a altos niveles de bilirubina. Toma valores valores $0$ o $1$, dependiendo de si el paciente presenta o no ictericia.|



## Problema de regresión Logística

Matemáticamente, este conjunto se define de la siguiente manera: 

$$\mathcal{D}=\left\{ \left(x_{i},y_{i}\right)\in\mathbb{R}^{p}\times\left\{ 0,1\right\} :i\in\left[m\right]\right\} $$.

El método de _regresión logística_ asume que $Pr\left[y_{i}\mid x_{i},\beta\right]\sim Bernoulli\left(\mu_{i}\right)$
con los siguientes supuestos sobre la media, $\mu_{i}$:

$$
\mu_{i}=\sigma\left(\beta^{T}x_{i}\right) \label{eq-3.1} \tag{1}
$$
$$
\sigma(z)=\left(1+\exp\left(-z\right)\right)^{-1} \label{eq-3.2} \tag{2}
$$

donde $\beta\in\mathbb{R}^{p}$. 


Dado lo anterior, nuestro problema es encontrar un modelo tal que $\hat{\beta}\in\mathbb{R}^{p}$ explica de la mejor manera posible a $\mathcal{D}$. 

Para lograr lo anterior, debemos estimar el conjunto de parámetros $\hat{\beta}$ para modelar $Pr\left[y\mid x,\hat{\beta}\right]$ y predecir la etiqueta $\hat{y}\in\left\{ 0,1\right\} $ de un nuevo
dato $x$ por medio de:

$$
\hat{y}=\begin{cases}
1 & si\,\sigma\left(\hat{\beta}^{T}x\right)\geq0.5\\
0 & si\,\sigma\left(\hat{\beta}^{T}x\right)<0.5
\end{cases}\label{eq-3.3} \tag{3}
$$

la función de pérdida que queremos minimizar en este problema corresponde a la _log-verosimilitud negativa_ , que está dada por:

$$
F(\beta):=LVN(\beta)=-\sum_{i=1}^{m}\left[y_{i}log\mu_{i}+(1-y_{i})log(1-\mu_{i})\right]\label{eq-3.4} \tag{4}
$$


Una vez planteado lo anterior, queremos encontrar $\hat{\beta}$ por medio de métodos numéricos de optimización de tal forma que se minimize ([4](#mjx-eqn-eq1)) para el conjunto de datos dado.


_En los siguientes fragmentos de código se realiza el planteamiento del problema, desde la importación de datos hasta el proceso de entrenamiento del modelo utilizando distintos algoritmos de optimización que se explican con brevedad._



---------------------

### Importación de datos

En esta sección se importa y transforma los datos, con el fin de obtener el conjunto $\mathcal{D}$.

In [1]:
# librerías
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
url="https://raw.githubusercontent.com/afcarl/ebola-imc-public/master/data/kenema/test/pres-kgh/imputation-50.csv"
df_raw=pd.read_csv(url,sep=",")

In [3]:
df_raw.head()
# df[df.isnull().any(axis=1)] - no hay NAs

,OUT,CT,AGE,TEMP,HEADCH,BLEED,DIARR,JAUN,VOMIT,PABD,WEAK
0,1,28.652450,42.0,36.3,0,0,1,0,0,1,1
1,1,25.736016,45.0,36.5,1,0,1,0,0,1,1
2,1,20.747653,65.0,38.0,1,0,0,0,0,0,0
3,1,22.736993,44.0,38.6,1,0,0,0,0,0,1
4,1,20.846284,11.0,38.4,1,0,0,0,1,0,1


In [4]:
# verificar tipo de variables 
df_raw.dtypes

OUT         int64
CT        float64
AGE       float64
TEMP      float64
HEADCH      int64
BLEED       int64
DIARR       int64
JAUN        int64
VOMIT       int64
PABD        int64
WEAK        int64
dtype: object

In [5]:
# resumen de las variables
df_raw.describe()

,OUT,CT,AGE,TEMP,HEADCH,BLEED,DIARR,JAUN,VOMIT,PABD,WEAK
count,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.0,106.000000,106.000000,106.000000
mean,0.764151,25.720411,34.102170,37.256604,0.603774,0.066038,0.405660,0.0,0.207547,0.273585,0.500000
std,0.426545,5.869164,17.382844,1.030767,0.491436,0.249528,0.493352,0.0,0.407477,0.447916,0.502375
min,0.000000,12.100000,0.830000,36.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,1.000000,22.149857,22.000000,36.300000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,1.000000,25.236301,35.500000,37.250000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.500000
75%,1.000000,28.680924,45.000000,38.225000,1.000000,0.000000,1.000000,0.0,0.000000,1.000000,1.000000
max,1.000000,39.799999,80.000000,39.900000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000



    
**Algunas observaciones sobre los datos**

- Se determinan 3 grupos de edades, utilizando el percetil 25 (22 años), percentil 50 (36 años) y percentil 75 (45 años)   
- Para este conjunto de datos la variable `JAUN` no tiene variabilidad, por lo tanto no es una variable, y se omite.

Dado lo anterior, se ajusta el set de datos

In [6]:
# ajustes en df_raw 
df_proc = df_raw
df_proc['INTER_AGE'] = "NA"

df_proc.dtypes

OUT            int64
CT           float64
AGE          float64
TEMP         float64
HEADCH         int64
BLEED          int64
DIARR          int64
JAUN           int64
VOMIT          int64
PABD           int64
WEAK           int64
INTER_AGE     object
dtype: object

In [7]:
# ajustes en df_raw 
df_proc = df_raw

# para la variable edad se crean cuatro categorías
age_p25 = math.ceil(df_proc['AGE'].quantile(.25))
age_p50 = math.ceil(df_proc['AGE'].quantile(.50))
age_p75 = math.ceil(df_proc['AGE'].quantile(.75))

df_proc['INTER_AGE'] = "NA"
df_proc.loc[(df_proc['AGE'] <= age_p25), 'INTER_AGE'] = 1
df_proc.loc[(df_proc['AGE'] > age_p25) & (df_proc['AGE'] <= age_p50), 'INTER_AGE'] = 2
df_proc.loc[(df_proc['AGE'] > age_p50) & (df_proc['AGE'] <= age_p75), 'INTER_AGE'] = 3
df_proc.loc[(df_proc['AGE'] > age_p75), 'INTER_AGE'] = 4

## one hot encoding
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df_proc[['INTER_AGE']]).toarray())
enc_df = enc_df.rename(columns={0: f"hasta{age_p25}", 1: f"entre{age_p25+1}y{age_p50}", 2: f"entre{age_p50+1}y{age_p75}", 3:f"mayor{age_p75}"})
# merge with main df bridge_df on key values
df_proc = df_proc.join(enc_df)

# se asignan como categoricas a las binarias, incluido el output
bin_vars = ['OUT', 'HEADCH', 'BLEED', 'DIARR', 'JAUN', 'VOMIT',
       'PABD', 'WEAK', 'INTER_AGE', f"hasta{age_p25}", f"entre{age_p25+1}y{age_p50}", f"entre{age_p50+1}y{age_p75}", f"mayor{age_p75}"]

#esta asignacion hace que genera problemas al evaluar el sigmoide
#for var in bin_vars:
#    df_proc[var] = df_proc[var].astype('category')
    
# se omiten las variables JAUN, AGE, INTER_AGE
del_vars = ["JAUN", "AGE", "INTER_AGE"]
for var in del_vars:
    df_proc = df_proc.drop(var, axis=1)    
    
# se comprueban los tipos de variable
df_proc.dtypes

OUT             int64
CT            float64
TEMP          float64
HEADCH          int64
BLEED           int64
DIARR           int64
VOMIT           int64
PABD            int64
WEAK            int64
hasta22       float64
entre23y36    float64
entre37y45    float64
mayor45       float64
dtype: object

In [8]:
df_proc

,OUT,CT,TEMP,HEADCH,BLEED,DIARR,VOMIT,PABD,WEAK,hasta22,entre23y36,entre37y45,mayor45
0,1,28.652450,36.3,0,0,1,0,1,1,0.0,0.0,1.0,0.0
1,1,25.736016,36.5,1,0,1,0,1,1,0.0,0.0,1.0,0.0
2,1,20.747653,38.0,1,0,0,0,0,0,0.0,0.0,0.0,1.0
3,1,22.736993,38.6,1,0,0,0,0,1,0.0,0.0,1.0,0.0
4,1,20.846284,38.4,1,0,0,1,0,1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,1,24.191797,36.4,0,0,1,1,1,1,0.0,0.0,1.0,0.0
102,1,20.846284,38.4,0,0,0,1,0,1,0.0,1.0,0.0,0.0
103,0,38.816561,36.0,0,0,0,0,0,0,1.0,0.0,0.0,0.0
104,1,21.960294,36.4,0,0,0,0,0,0,0.0,1.0,0.0,0.0


### Planteamiento del problema de regresión

A continuación se plantea el código que computa las ecuaciones ([1](#mjx-eqn-eq1)), ([2](#mjx-eqn-eq1)) y ([4](#mjx-eqn-eq1)), planteadas inicialmente.

In [9]:
def sigmoide(z):
    '''
    Devuelve el sigmoide de un vector
        ** Parámetros:
            - z (vec): vector numérico de m entradas
        ** Salidas
            - (float64) valor entre -1 y 1
    '''
    return 1/(1+ np.exp(-z))
    
def calc_mu(X,beta):
    '''
    Calcula la media para una variable aleatoria con distribución bernoulli.
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - beta (vec): vector de p entradas
        ** Salidas
            - mu (vec): vector de m entradas
    '''
    a = np.matmul(beta,np.transpose(X))
    mu = sigmoide(a)

    return mu
    
def f(X,y,beta):
    '''
    Función que computa la log-verosimilitud negativa
    ** Parámetros:
        - X (mat): matriz de mxp entradas
        - y (vec): vector de de m entradas de la variable output
        - beta (vec): vector de p entradas
    ** Salidas
        - lvn (int): log-verosimilitud negativa
    '''
    prob = calc_mu(X,beta)
    # log-verosimilitud negativa 
    lvn = -sum(y*np.log(prob)+(1-y)*(np.log(1-prob)))
    return lvn

Reescribiendo la ecuación de la función de pérdida ([4](#mjx-eqn-eq1)), tenemos:

$$F(\beta)=- \sum_{i=1}^{m}[y_i log\mu_i + (1-y_i)log(1-\mu_i)]$$

Las expresiones correspondientes al gradiente y a la matriz hessiana asociados a este problema, se plantean a continuación:

\begin{align}
\nabla F(\beta) & =\frac{d}{d\beta}F(\beta)\nonumber \\
 & =\sum_{i}\left(\mu_{i}-y_{i}\right)x_{i}\nonumber \\
 & =\boldsymbol{X}^{T}\left(\boldsymbol{\mu}-\boldsymbol{y}\right)\label{eq:gradient}
\end{align}

Por otro lado, la ecuación que describe la matrix Hessiana es la siguiente:

\begin{align}
\nabla^{2}F(\beta) & =\frac{d}{d\beta}\nabla F\left(\beta\right)^{T}\nonumber \\
 & =\sum_{i}\left(\nabla_{\beta}\mu_{i}\right)x_{i}^{T}\nonumber \\
 & =\sum_{i}\mu_{i}\left(1-\mu_{i}\right)x_{i}x_{i}^{T}\nonumber \\
 & =\boldsymbol{X^{T}SX}\label{eq:hessian}
\end{align}

donde $\boldsymbol{S}\triangleq diag\left(\mu_{i}\left(1-\mu_{i}\right)\right)$.
Como es resaltado por Murphy (2012), es definida positiva, lo que implica que ([4](#mjx-eqn-eq1)) es convexa
y tiene un mínimo global.

In [10]:
def gradiente_f(X,y,beta):
    '''
    Calcula el gradiente asociado la log-verosimilitud negativa del problema de regresión logística
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - beta (vec): vector de p entradas
        ** Salidas
            - grad (vec): vector de m entradas
    '''
    mu=calc_mu(X,beta)    
    grad = np.matmul(np.transpose(X), mu-y)    
    return grad


def hessiana_f(X,y,beta):
    '''
    Calcula la matriz Hessiana asociada a la log-verosimilitud negativa del problema de regresión logística
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - beta (vec): vector de p entradas
        ** Salidas
            - grad (vec): vector de m entradas
    '''
    mu=calc_mu(X,beta)
    S=np.diag(mu*(1-mu))
    hes=np.matmul(np.transpose(X),np.matmul(S,X))
    return hes

In [11]:
def normalize(x):
    return x/np.sqrt(sum(x*x))

In [12]:
def clasifica(X, beta_hat):
    mu=calc_mu(X,beta_hat)
    yhat=mu
    yhat[mu<.5]=0
    yhat[mu>=.5]=1
    return yhat

In [13]:
def descent_direction(X, y, beta, method="max"):
    '''
    This function determines the direction of the descent pk=-inv(Bk) grad f
    '''
    if(method=="max"):
        pk=gradiente_f(X,y,beta)
    
    elif(method=="newton"):
        grad=gradiente_f(X,y,beta)
        hess=hessiana_f(X,y,beta)
        inv_hess=np.linalg.inv(hess)
        pk=np.matmul(inv_hess,grad)
        #pk=np.linalg.solve(hess,grad)
    
    return normalize(pk)


In [14]:
def gradient_descent(X, y,lr=.1, tol=10**(-6), max_iter=10**5, method="max"):
    '''
    Devuelve vector de parámetros beta (px1) resultante del proceso de optimización por descenos de gradiente
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - lr (float64): tasa de aprendizaje
            - tol (float64): criterio de convergencia
            - max_iter (int): número máximo de iteraciones
            - method (str): método que determina la dirección de descenso
                opciones:
                    - max: método de descenso
                    - newton: método de Newton
                    - 
            
        ** Salidas
            - beta_new (vec): vector de p entradas con parámetros que minimizan la función de pérdida
    '''
    
    #inicializa
    iteraciones=0
    
    #inicializamos beta aleatoria
    beta=np.random.uniform(0,1,X.shape[1])
    
    #primera iteracion
    pk = descent_direction(X, y, beta, method)
    beta_new= beta - lr*pk
    
    #condición de paro.
    #El cambio total es menor que la tolerancia
    while ((abs(f(X,y,beta) - f(X,y,beta_new))>tol) & (iteraciones<max_iter)):
        iteraciones+=1 #contador de ciclo
        
        beta = beta_new
        pk = descent_direction(X,y,beta,method)
        beta_new = beta - lr*pk
    
    print("iteraciones=",iteraciones)
    return beta_new

**Nota:** Implementación tomando datos anteriores - se debe corregir la matriz de diseño

In [15]:
url="https://raw.githubusercontent.com/afcarl/ebola-imc-public/master/data/kenema/test/pres-kgh/imputation-50.csv"
df=pd.read_csv(url,sep=",")

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

data =df_proc.to_numpy()
y = data[:,0]
X = data[:,1:]
x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=.2)

#scale data
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [18]:
beta_hat=gradient_descent(x_train,y_train)
yhat=clasifica(x_test,beta_hat)

print("beta_hat=", beta_hat)
print("Error de clasificacion=",round(100*sum(abs(y_test-yhat))/len(yhat),2),"%")

iteraciones= 439
beta_hat= [-21.06989336  28.77241483  -6.92602042   0.38290737  -0.45738984
   8.15939103  -1.23567437   7.16971246   6.82020059   8.36579298
  11.51117882  13.0196031 ]
Error de clasificacion= 9.09 %


/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in multiply


In [19]:
# Método de Newton (no está funcionando)
beta_hat=gradient_descent(x_train,y_train, method="newton")
yhat=clasifica(x_test,beta_hat)

print("beta_hat=", beta_hat)
print("Error de clasificacion=",round(100*sum(abs(y_test-yhat))/len(yhat),2),"%")

iteraciones= 427
beta_hat= [-21.26937682  28.30946471  -6.47576423   4.80268579  -0.51982638
   8.40999312  -1.53582079   7.09875603   7.02324047   8.56310126
  11.483358    13.20805439]
Error de clasificacion= 9.09 %


/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in multiply
